<a href="https://colab.research.google.com/github/abhaysrivastav/finetuning-methods/blob/main/Chapter2_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Article Translation using Few-Shot Learning with Flan-T5

In [ ]:
!pip install --upgrade transformers tensorflow datasets

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [ ]:

from datasets import load_dataset

# Load the Dailymail dataset
dataset = load_dataset('abisee/cnn_dailymail', '3.0.0')
print(dataset['train'][0])


## Selecting Dialogues and Creating Few-Shot Examples

In [ ]:

# Selecting three dialogues from the DailyMail dataset and creating manual translations
articles = [
    {
        "article": """
As he turns 18, Harry Potter star Daniel Radcliffe will gain access to a £20 million ($41.1 million) fortune. Despite this significant wealth, Radcliffe insists the money will not lead him astray. He has no plans to spend lavishly on fast cars, alcohol, or celebrity parties. Instead, he prefers to buy books, CDs, and DVDs. Details of his birthday celebration remain private, though Radcliffe mentioned he would have a party but hopes it won't attract media attention. He emphasizes his determination to stay grounded and avoid the typical pitfalls associated with young stars. Radcliffe's earnings from the first five Harry Potter films have been in a trust fund until now. His portrayal in "Harry Potter and the Order of the Phoenix" is breaking box office records. He will reprise his role in the final two films of the series. Beyond Potter, Radcliffe has filmed a TV movie called "My Boy Jack" and will appear in "December Boys," an Australian film. He also made his stage debut in "Equus" earlier this year. As he anticipates increased media scrutiny with his legal adulthood, Radcliffe remains focused on maintaining his privacy and composure, demonstrating a mature approach to his fame and fortune.
""",
        "translation": """
Al cumplir 18 años, la estrella de Harry Potter, Daniel Radcliffe, tendrá acceso a una fortuna de £20 millones ($41.1 millones). A pesar de esta considerable riqueza, Radcliffe insiste en que el dinero no lo desviará de su camino. No tiene planes de gastar lujosamente en autos rápidos, alcohol o fiestas de celebridades. En cambio, prefiere comprar libros, CDs y DVDs. Los detalles de su celebración de cumpleaños se mantienen privados, aunque Radcliffe mencionó que tendrá una fiesta, pero espera que no atraiga la atención de los medios. Enfatiza su determinación de mantenerse con los pies en la tierra y evitar los típicos escollos asociados con las jóvenes estrellas. Las ganancias de Radcliffe de las primeras cinco películas de Harry Potter han estado en un fondo fiduciario hasta ahora. Su interpretación en "Harry Potter y la Orden del Fénix" está batiendo récords de taquilla. Repetirá su papel en las dos últimas películas de la serie. Más allá de Potter, Radcliffe ha filmado una película para televisión llamada "My Boy Jack" y aparecerá en "December Boys," una película australiana. También hizo su debut en el escenario en "Equus" a principios de este año. Mientras anticipa un mayor escrutinio mediático con su adultez legal, Radcliffe se mantiene enfocado en mantener su privacidad y compostura, demostrando un enfoque maduro hacia su fama y fortuna.
"""
    },
    {
        "article": """
The ninth floor of the Miami-Dade pretrial detention facility, known as the "forgotten floor," houses inmates with severe mental illnesses until they are ready to appear in court. These inmates often face drug charges or charges of assaulting an officer, which Judge Steven Leifman describes as "avoidable felonies" often resulting from confrontations with police. These interactions exacerbate the inmates' mental conditions, making them more paranoid and less compliant. Despite being severely mentally disturbed, these inmates do not receive adequate help while in jail. Judge Leifman, an advocate for justice and mental health, highlights the overwhelming volume of mentally ill inmates in Miami-Dade county jails, estimating about one-third of the jail population. During a tour of the facility, the harsh conditions became evident: inmates wear sleeveless robes designed to prevent self-harm, and the environment is stark, with glaring lights and loud, tiny cells. Some prisoners were observed naked or in sleeveless robes, shouting delusional statements. Leifman explains that these inmates often cycle through the system, stabilizing temporarily in mental hospitals before returning to jail. Historically, people with mental illnesses were confined to jails or inadequate hospitals, leading to many becoming homeless without proper treatment. To address this, starting in 2008, many inmates who would have been sent to the "forgotten floor" will be directed to a new mental health facility aimed at long-term treatment. Leifman believes this approach will benefit patients, relieve families, save state funds, and ultimately serve justice.
""",
        "translation": """
El noveno piso de la instalación de detención preventiva de Miami-Dade, conocido como el "piso olvidado," alberga a reclusos con enfermedades mentales graves hasta que estén listos para comparecer ante el tribunal. Estos reclusos a menudo enfrentan cargos de drogas o de agredir a un oficial, que el juez Steven Leifman describe como "delitos evitables" que a menudo resultan de enfrentamientos con la policía. Estas interacciones exacerban las condiciones mentales de los reclusos, haciéndolos más paranoicos y menos obedientes. A pesar de estar gravemente perturbados mentalmente, estos reclusos no reciben la ayuda adecuada mientras están en la cárcel. El juez Leifman, un defensor de la justicia y la salud mental, destaca el volumen abrumador de reclusos con enfermedades mentales en las cárceles del condado de Miami-Dade, estimando aproximadamente un tercio de la población carcelaria. Durante un recorrido por la instalación, las duras condiciones se hicieron evidentes: los reclusos llevan túnicas sin mangas diseñadas para prevenir autolesiones, y el ambiente es austero, con luces deslumbrantes y celdas pequeñas y ruidosas. Algunos prisioneros fueron observados desnudos o con túnicas sin mangas, gritando declaraciones delirantes. Leifman explica que estos reclusos a menudo circulan por el sistema, estabilizándose temporalmente en hospitales mentales antes de regresar a la cárcel. Históricamente, las personas con enfermedades mentales fueron confinadas en cárceles o hospitales inadecuados, lo que llevó a muchos a quedar sin hogar sin recibir el tratamiento adecuado. Para abordar esto, a partir de 2008, muchos reclusos que habrían sido enviados al "piso olvidado" serán dirigidos a una nueva instalación de salud mental orientada al tratamiento a largo plazo. Leifman cree que este enfoque beneficiará a los pacientes, aliviará a las familias, ahorrará fondos estatales y, en última instancia, hará justicia.
"""
    },
    {
        "article": """
When the Minneapolis bridge collapsed, drivers on the bridge recounted harrowing survival stories. Gary Babineau, one of the survivors, described how the entire bridge fell, leading to a 30- to 35-foot free fall for him. Injured but mobile, he immediately helped rescue children from a nearby school bus amidst the chaos of cars in the water and on fire. Meanwhile, Dr. John Hink, an emergency room physician, rushed to the scene within 15 minutes and began assisting with the rescue efforts, which were organized despite the disaster's rapid onset. Volunteers, EMTs, and officials managed to get 55 people into ambulances in under two hours, often using pickup trucks to reach and transport the injured from the riverbank. Babineau's truck was precariously balanced on a broken section of the bridge, and he helped a woman and children to safety despite lacking rescue training. Melissa Hughes, another survivor, described the surreal experience of the bridge collapsing under her, and how a pickup landed on her car without causing her injury. Babineau recounted thinking he was going to die as his truck folded in half during the collapse. Similarly, Bernie Toivonen's vehicle ended up tilted at a 45-degree angle, and he thought he was going to die. After the bridge settled, Toivonen was relieved to find himself alive. These personal accounts highlight the dramatic and chaotic nature of the bridge collapse and the immediate, life-saving responses from ordinary people and first responders.
""",
        "translation": """
Cuando el puente de Minneapolis colapsó, los conductores en el puente relataron historias desgarradoras de supervivencia. Gary Babineau, uno de los sobrevivientes, describió cómo todo el puente se cayó, llevándolo a una caída libre de 30 a 35 pies. Herido pero móvil, ayudó inmediatamente a rescatar a los niños de un autobús escolar cercano en medio del caos de autos en el agua y en llamas. Mientras tanto, el Dr. John Hink, un médico de la sala de emergencias, se apresuró al lugar en 15 minutos y comenzó a ayudar con los esfuerzos de rescate, que se organizaron a pesar del rápido desastre. Voluntarios, técnicos de emergencias médicas y otros funcionarios lograron llevar a 55 personas a las ambulancias en menos de dos horas, a menudo utilizando camionetas para alcanzar y transportar a los heridos desde la orilla del río. El camión de Babineau estaba precariamente equilibrado en una sección rota del puente, y ayudó a una mujer y a los niños a ponerse a salvo a pesar de no tener entrenamiento de rescate. Melissa Hughes, otra sobreviviente, describió la surrealista experiencia del colapso del puente bajo ella y cómo una camioneta aterrizó en su coche sin causarle daño. Babineau relató pensar que iba a morir mientras su camión se doblaba por la mitad durante el colapso. De manera similar, el vehículo de Bernie Toivonen terminó inclinado en un ángulo de 45 grados, y pensó que iba a morir. Después de que el puente se asentó, Toivonen se sintió aliviado al descubrir que estaba vivo. Estos relatos personales destacan la naturaleza dramática y caótica del colapso del puente y las respuestas inmediatas y salvadoras de personas comunes y socorristas.
"""
    }
]

for idx, example in enumerate(articles):
    print(f"Article {idx + 1}:")
    print(example["article"])
    print(f"Translation {idx + 1}:")
    print(example["translation"])
    print()


## Constructing the Few-Shot Learning Prompt

In [ ]:

# Constructing the few-shot learning prompt with the selected dialogues and their translations
few_shot_prompt = ""

for example in articles[:2]:
    few_shot_prompt += f"Translate this text from English to Spanish:\n\n Text: \n\n {example['article']} \n\n Translation: \n\n {example['translation']} \n\n "

# Print the few-shot prompt
print(few_shot_prompt)


## Specifying the Translation Task

In [ ]:

# New dialogue to translate
new_article = dataset['test'][0]['article']

# Adding the new dialogue to the prompt
prompt_with_new_article = few_shot_prompt + f"\n Translate this text from English to Spanish: \n\n Text: \n\n {new_article} \n\n Translation: "

# Print the final prompt
print(prompt_with_new_article)


## Implementing and Testing the Prompt

In [ ]:

# Generate inputs for the prompt
inputs = tokenizer(prompt_with_new_article, return_tensors="tf")

# Generate outputs
outputs = model.generate(inputs["input_ids"], max_length=500)

# Decode and print the translation
translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Translation:", translation)


## Evaluating and Iterating

In [ ]:

# Analyze the effectiveness of the translation
print("New Article:", new_article)
print("Translation:", translation)

# Here you can refine your prompt based on the translation results
# For example, if the translation lacks context, you might add more detailed examples or adjust the prompt wording
